In [4]:
import pandas as pd
import numpy as np
import io
import re
import base64
from urllib.parse import quote as urlquote

def dms_to_decimal(dms):
    match = re.search(r"(\d+)�(\d+)'([\d\.]+)\"([NSEW])", dms)
    if match:
        degrees, minutes, seconds, direction = match.groups()
        decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        return None


def prepare_df(path):
    columnas = ["Fecha", "Hora","Latitud","Longitud"]
    path = f'{path}'
    if "csv" in path:
        sabana = pd.read_csv(path, encoding='utf-8')
    elif "xlsx" in path:
        sabana = pd.read_excel(path)
    else:
        return None
    
    for idx, column_name in enumerate(sabana):
        column = column_name.lower()
        if "fecha de la comunicación" == column:
            sabana['Fecha'] = sabana[column_name]
        if "fecha" == column:
            sabana['Fecha'] = sabana[column_name]
        if "hora de la comunicación" == column:
            sabana['Hora'] = sabana[column_name]
        if "hora" == column:
            sabana['Hora'] = sabana[column_name]
        if "latitud" == column:
            sabana['Latitud'] = sabana[column_name]
        if "longitud" == column:
            sabana['Longitud'] = sabana[column_name]
        if "ubicacion geografica (latitud / longitud)" == column:
            sabana['Latitud'] = sabana.iloc[:,idx]
            sabana['Longitud'] = sabana.iloc[:,idx+1]
            sabana['Latitud'] = sabana['Latitud'].apply(dms_to_decimal)
            sabana['Longitud'] = sabana['Longitud'].apply(dms_to_decimal)
    sabana = sabana.drop_duplicates(subset=columnas)
    sabana = sabana.dropna(subset=["Latitud","Longitud"])
    return sabana[columnas]

df = prepare_df("5585996167.csv")
#df = prepare_df("9844706032.csv")
#df = prepare_df("Copia de 9844706032.xlsx")
#df = prepare_df("9981656564.csv")
#df = prepare_df("9988741541.csv")
#df = prepare_df("9988744863.csv")
df


,Fecha,Hora,Latitud,Longitud
0,10/10/2023,00:00:48,19.416111,-99.240000
1,10/10/2023,00:01:31,19.416111,-99.240000
2,10/10/2023,00:02:14,19.416111,-99.240000
3,10/10/2023,00:02:56,19.416111,-99.240000
4,10/10/2023,00:03:39,19.416111,-99.240000
...,...,...,...,...
44443,10/01/2024,20:13:00,19.383333,-99.274167
44445,10/01/2024,20:28:00,19.380833,-99.274444
44451,10/01/2024,20:46:56,19.430189,-99.287319
44455,10/01/2024,21:45:11,19.394722,-99.281389


In [3]:
print("""""")